### Prof. Pedram Jahangiry

You need to make a copy to your own Google drive if you want to edit the original notebook! Start by opening this notebook on Colab 👇

<a href="https://colab.research.google.com/github/PJalgotrader/Deep_forecasting-USU/blob/main/Lectures%20and%20codes/Module%204-%20ARIMA/Module4-ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



![logo](https://upload.wikimedia.org/wikipedia/commons/4/44/Huntsman-Wordmark-with-USU-Blue.gif#center)


## 🔗 Links

[![linkedin](https://img.shields.io/badge/LinkedIn-0A66C2?style=for-the-badge&logo=linkedin&logoColor=white)](https://www.linkedin.com/in/pedram-jahangiry-cfa-5778015a)

[![Youtube](https://img.shields.io/badge/youtube_channel-1DA1F2?style=for-the-badge&logo=youtube&logoColor=white&color=FF0000)](https://www.youtube.com/channel/UCNDElcuuyX-2pSatVBDpJJQ)

[![Twitter URL](https://img.shields.io/twitter/url/https/twitter.com/PedramJahangiry.svg?style=social&label=Follow%20%40PedramJahangiry)](https://twitter.com/PedramJahangiry)


---


# Module 4: ARIMA models

In this module, we cover the basics of ARIMA (AutoRegressive Integrated Moving Average) models, a commonly used statistical method for time series forecasting. Our focus will be on understanding the underlying concepts and components of ARIMA models, as well as how to implement them in practice.

We start by discussing the properties of time series data and the need for a statistical model to capture its behavior. Next, we delve into the components of ARIMA models - autoregression, integration, and moving average - and their role in capturing patterns and making predictions based on past values.

We also cover the process of making time series data stationary and selecting the appropriate ARIMA parameters (p, d, q) based on autocorrelation and partial autocorrelation plots. Finally, we demonstrate how to fit ARIMA models to time series data and make predictions using Python packages such as sktime and PyCaret.

Documentation:

1. **PyCaret**: https://pycaret.readthedocs.io/en/latest/index.html PyCaret3.0
2. **sktime** : https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.arima.ARIMA.html

# Installation

Follow the steps here: https://pycaret.gitbook.io/docs/get-started/installation


In [ ]:
#only if you want to run it in Google Colab:
# for this chapter, we can install the light version of PyCaret as below.

#!pip install pycaret

In [2]:
# if you got a warning that you need to "RESTART RUNTIME", go ahead and press that button.

# let's double ckeck the Pycaret version:
from pycaret.utils import version
version()

'3.3.2'

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Importing Dataset

In [4]:
from pycaret.datasets import get_data
airline = get_data('airline')

Period
1949-01    112.0
1949-02    118.0
1949-03    132.0
1949-04    129.0
1949-05    121.0
Freq: M, Name: Number of airline passengers, dtype: float64

In [5]:
# or alternatively,
df = pd.read_csv("https://raw.githubusercontent.com/PJalgotrader/Deep_forecasting-USU/main/data/airline_passengers.csv", index_col="Month")
df.head()

,Passengers
Month,
1949-01,112
1949-02,118
1949-03,132
1949-04,129
1949-05,121


In [6]:
# if you are working with Pandas, your first job should be changing the type of the index to datetime and then to period! This is a compatibility issue with other packages.
df.index

Index(['1949-01', '1949-02', '1949-03', '1949-04', '1949-05', '1949-06',
       '1949-07', '1949-08', '1949-09', '1949-10',
       ...
       '1960-03', '1960-04', '1960-05', '1960-06', '1960-07', '1960-08',
       '1960-09', '1960-10', '1960-11', '1960-12'],
      dtype='object', name='Month', length=144)

In [7]:
df.index = pd.to_datetime(df.index).to_period('M')
df.index

PeriodIndex(['1949-01', '1949-02', '1949-03', '1949-04', '1949-05', '1949-06',
             '1949-07', '1949-08', '1949-09', '1949-10',
             ...
             '1960-03', '1960-04', '1960-05', '1960-06', '1960-07', '1960-08',
             '1960-09', '1960-10', '1960-11', '1960-12'],
            dtype='period[M]', name='Month', length=144)

Setting up PyCaret Experiment:

In [8]:
from pycaret.time_series import *

In [9]:
exp = TSForecastingExperiment()
exp.setup(data = df, target='Passengers' ,  fh = 12, coverage=0.95)

,Description,Value
0,session_id,6095
1,Target,Passengers
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(144, 1)"
5,Transformed data shape,"(144, 1)"
6,Transformed train set shape,"(132, 1)"
7,Transformed test set shape,"(12, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [10]:
exp.check_stats()

,Test,Test Name,Data,Property,Setting,Value
0,Summary,Statistics,Transformed,Length,,144.0
1,Summary,Statistics,Transformed,# Missing Values,,0.0
2,Summary,Statistics,Transformed,Mean,,280.298611
3,Summary,Statistics,Transformed,Median,,265.5
4,Summary,Statistics,Transformed,Standard Deviation,,119.966317
5,Summary,Statistics,Transformed,Variance,,14391.917201
6,Summary,Statistics,Transformed,Kurtosis,,-0.364942
7,Summary,Statistics,Transformed,Skewness,,0.58316
8,Summary,Statistics,Transformed,# Distinct Values,,118.0
9,White Noise,Ljung-Box,Transformed,Test Statictic,"{'alpha': 0.05, 'K': 24}",1606.083817


* The Ljung-Box test is a statistical test that is commonly used to check whether there are any autocorrelations in a time series. Specifically, it tests the null hypothesis that the autocorrelations of the time series data for lags 1 through K are all equal to zero.
* ADF null hypothesis: The time series has a unit root, meaning it is non-stationary. small p-value is in favor of stationarity.
* KPSS null hypothesis: The time series is stationary around a deterministic trend (or simply stationary if no trend is included in the test equation). Large p-value is in favor of stationarity
* Shapiro-Wilk null hypothesis: The sample comes from a normally distributed population. Large p-value is in favor of normality.

In [11]:
exp.plot_model(plot='train_test_split')


---
---
# ARIMA models:

## Selecting p and q:
Remember, to select p and q, we must first make the data astationary! That's why we will plot the difference model.

**Difference plotting using orders:**

find the data_kwargs here: https://github.com/pycaret/pycaret/blob/master/pycaret/time_series/forecasting/functional.py

In [57]:
exp.plot_model(plot="diff", data_kwargs={"order_list": [1,12], "acf": True, "pacf": True})


**Difference Plot Using Lags:**

For example, given a timeseries with monthly periodicity, using lags=[1, 12] corresponds to applying a standard first difference to handle trend, and followed by a seasonal difference (at lag 12) to attempt to account for seasonal dependence.

In [58]:
exp.plot_model(plot="diff", data_kwargs={"lags_list": [[1,12]], "acf": True, "pacf": True})


Based on the above plot, it seems that SARIMA(1,1,1)(1,1,2,12) is a good start. (another good candidate might be SARIMA(1,1,1)(1,1,1,12)

However, to compare the performance of different components of ARIMA model, let's construct 4 more models. So we have 5 models to compare + two bench marks!
1. AR(1)
2. MA(1)
3. ARIMA(1,1,1)
4. SARIMA(1,1,1)(1,1,2,12)
5. Random walk: ARIMA(0,1,0) with no constant
6. Random walk with drift: ARIMA(0,1,0) with constant

---
### ARIMA

In [84]:
exp.models()

,Name,Reference,Turbo
ID,,,
naive,Naive Forecaster,sktime.forecasting.naive.NaiveForecaster,True
grand_means,Grand Means Forecaster,sktime.forecasting.naive.NaiveForecaster,True
snaive,Seasonal Naive Forecaster,sktime.forecasting.naive.NaiveForecaster,True
polytrend,Polynomial Trend Forecaster,sktime.forecasting.trend._polynomial_trend_for...,True
arima,ARIMA,sktime.forecasting.arima.ARIMA,True
auto_arima,Auto ARIMA,sktime.forecasting.arima.AutoARIMA,True
exp_smooth,Exponential Smoothing,sktime.forecasting.exp_smoothing.ExponentialSm...,True
ets,ETS,sktime.forecasting.ets.AutoETS,True
theta,Theta Forecaster,sktime.forecasting.theta.ThetaForecaster,True


In [85]:
ar1 = exp.create_model('arima', order = (1,0,0), seasonal_order=(0,0,0,12), with_intercept=True, cross_validation=False)
# by default, "with_intercept=True", we don't need to add it mannually.

,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
Test,3.3760,3.7460,102.7986,129.4174,0.1972,0.2297,-2.0236


In [86]:
ma1 = exp.create_model('arima', order = (0,0,1), seasonal_order=(0,0,0,12), with_intercept= True, cross_validation=False)

,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
Test,6.6833,6.4226,203.5061,221.8905,0.4119,0.5336,-7.8881


In [87]:
arima111= exp.create_model('arima', order = (1,1,1), seasonal_order=(0,0,0,12) , with_intercept= True, cross_validation=False)

,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
Test,1.9990,2.3844,60.8693,82.3772,0.1166,0.1256,-0.2250


In [90]:
sarima111112= exp.create_model('arima', order = (1,1,1), seasonal_order=(1,1,2,12) , with_intercept= True, cross_validation=False)

,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
Test,0.4774,0.5456,14.5361,18.8508,0.0323,0.0314,0.9359


In [91]:
rw= exp.create_model('arima', order = (0,1,0), seasonal_order=(0,0,0,12) , with_intercept= False, cross_validation=False)
# remember, Random walk is equivalent to naive forecaster. So this code also works: exp.create_model('naive', cross_validation=False)

,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
Test,2.4959,2.9807,76.0000,102.9765,0.1425,0.1612,-0.9143


In [92]:
rwwd= exp.create_model('arima', order = (0,1,0), seasonal_order=(0,0,0,12) , with_intercept= True, cross_validation=False)

,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
Test,2.1776,2.6822,66.3079,92.6664,0.1242,0.1381,-0.5502


In [93]:
my_models = [rw, rwwd, ar1, ma1, arima111, sarima111112]
my_model_lables = ['Random Walk', 'Random Walk with drift', 'AR(1)', 'MA(1)', 'ARIMA(1,1,1)', 'SARIMA(1,1,1)(1,1,2,12)']

In [96]:
exp.compare_models(my_models, cross_validation=False)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
5,ARIMA,0.4774,0.5456,14.5361,18.8508,0.0323,0.0314,0.9359,1.5800
4,ARIMA,1.9990,2.3844,60.8693,82.3772,0.1166,0.1256,-0.2250,0.0500
1,ARIMA,2.1776,2.6822,66.3079,92.6664,0.1242,0.1381,-0.5502,0.0100
0,ARIMA,2.4959,2.9807,76.0000,102.9765,0.1425,0.1612,-0.9143,0.0100
2,ARIMA,3.3760,3.7460,102.7986,129.4174,0.1972,0.2297,-2.0236,0.0200
3,ARIMA,6.6833,6.4226,203.5061,221.8905,0.4119,0.5336,-7.8881,0.0200


ARIMA(order=(1, 1, 1), seasonal_order=(1, 1, 2, 12))

---


In [97]:
sarima111112.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                        SARIMAX Results                                        
===============================================================================================
Dep. Variable:                                       y   No. Observations:                  132
Model:             SARIMAX(1, 1, 1)x(1, 1, [1, 2], 12)   Log Likelihood                -442.065
Date:                                 Wed, 06 Nov 2024   AIC                            898.130
Time:                                         15:39:20   BIC                            917.584
Sample:                                     01-31-1949   HQIC                           906.030
                                          - 12-31-1959                                         
Covariance Type:                                   opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0008      0.068      0.012      0.991      -0.133       0.135
ar.L1         -0.3869      0.383     -1.009      0.313      -1.139       0.365
ma.L1          0.0951      0.393      0.242      0.809      -0.676       0.866
ar.S.L12       0.9983      0.146      6.831      0.000       0.712       1.285
ma.S.L12      -1.3094      1.527     -0.858      0.391      -4.302       1.683
ma.S.L24       0.3360      0.444      0.756      0.450      -0.535       1.207
sigma2        85.0860    119.551      0.712      0.477    -149.230     319.402
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):                 0.74
Prob(Q):                              0.99   Prob(JB):                         0.69
Heteroskedasticity (H):               1.64   Skew:                            -0.10
Prob(H) (two-sided):                  0.12   Kurtosis:                         3.33
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

**Exercise**: write down the equation for the SARIMA model?

---
#### Plotting models

In [99]:
exp.plot_model(sarima111112  , plot='forecast', data_kwargs={'fh':24, 'labels':['SARIMA(1,1,1)(1,1,2,12)']})

In [100]:
exp.plot_model(sarima111112, plot='diagnostics')

In [101]:
# let's test the stationarity of the residuals for the SARIMA(1,1,0)(1,1,1,12) model:
exp.check_stats(sarima111112, test = 'adf')


,Test,Test Name,Data,Property,Setting,Value
0,Stationarity,ADF,Residual,Stationarity,{'alpha': 0.05},True
1,Stationarity,ADF,Residual,p-value,{'alpha': 0.05},0.0
2,Stationarity,ADF,Residual,Test Statistic,{'alpha': 0.05},-11.786369
3,Stationarity,ADF,Residual,Critical Value 1%,{'alpha': 0.05},-3.481682
4,Stationarity,ADF,Residual,Critical Value 5%,{'alpha': 0.05},-2.884042
5,Stationarity,ADF,Residual,Critical Value 10%,{'alpha': 0.05},-2.57877


In [102]:
my_models

[ARIMA(order=(0, 1, 0), seasonal_order=(0, 0, 0, 12), with_intercept=False),
 ARIMA(order=(0, 1, 0), seasonal_order=(0, 0, 0, 12)),
 ARIMA(seasonal_order=(0, 0, 0, 12)),
 ARIMA(order=(0, 0, 1), seasonal_order=(0, 0, 0, 12)),
 ARIMA(order=(1, 1, 1), seasonal_order=(0, 0, 0, 12)),
 ARIMA(order=(1, 1, 1), seasonal_order=(1, 1, 2, 12))]

In [103]:
my_model_lables

['Random Walk',
 'Random Walk with drift',
 'AR(1)',
 'MA(1)',
 'ARIMA(1,1,1)',
 'SARIMA(1,1,1)(1,1,2,12)']

In [104]:
exp.plot_model(ar1, plot='forecast', data_kwargs={'fh':36})

In [105]:
exp.plot_model(ma1, plot='forecast', data_kwargs={'fh':36})

In [106]:
exp.plot_model(my_models, plot='forecast', data_kwargs={'fh':36, 'labels':my_model_lables})

In [107]:
exp.plot_model(my_models, plot='insample', data_kwargs={'labels':my_model_lables})

---
### Auto ARIMA
https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.arima.AutoARIMA.html

Wrapper of the pmdarima implementation of fitting Auto-(S)ARIMA(X) models. The auto-ARIMA algorithm seeks to identify the most optimal parameters for an ARIMA model, settling on a single fitted ARIMA model. This process is based on the commonly-used R function, forecast::auto.arima.

Auto-ARIMA works by conducting differencing tests (i.e., Kwiatkowski–Phillips–Schmidt–Shin, Augmented Dickey-Fuller or Phillips–Perron) to determine the order of differencing, d, and then fitting models within ranges of defined start_p, max_p, start_q, max_q ranges. If the seasonal optional is enabled, auto-ARIMA also seeks to identify the optimal P and Q hyper-parameters after conducting the Canova-Hansen to determine the optimal order of seasonal differencing, D.

In order to find the best model, auto-ARIMA optimizes for a given information_criterion, one of (‘aic’, ‘aicc’, ‘bic’, ‘hqic’, ‘oob’) (Akaike Information Criterion, Corrected Akaike Information Criterion, Bayesian Information Criterion, Hannan-Quinn Information Criterion, or “out of bag”–for validation scoring–respectively) and returns the ARIMA which minimizes the value


In [108]:
# finding the best model based on information criterion bic
auto_arima = exp.create_model('auto_arima', cross_validation=False, information_criterion='aic', start_p=0, start_q=0, max_p=2, max_q=2, seasonal=True, stepwise=False)

# stepwise=True is the default value, but we set it to False to make sure that the model is not limited to the stepwise search. the stepwise search is faster but not necessarily the best.

,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
Test,0.3764,0.4599,11.4614,15.8874,0.0239,0.0239,0.9544


In [109]:
auto_arima_fast = exp.create_model('auto_arima', cross_validation=False, information_criterion='aic', start_p=0, start_q=0, max_p=5, max_q=5, seasonal=True, stepwise=True)

,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
Test,0.4893,0.5365,14.8982,18.5365,0.0310,0.0309,0.9380


In [110]:
# getting auto_arima's hyperparameters
auto_arima.get_params()

{'D': None,
 'alpha': 0.05,
 'concentrate_scale': False,
 'd': None,
 'enforce_invertibility': True,
 'enforce_stationarity': True,
 'error_action': 'warn',
 'hamilton_representation': False,
 'information_criterion': 'aic',
 'max_D': 1,
 'max_P': 2,
 'max_Q': 2,
 'max_d': 2,
 'max_order': 5,
 'max_p': 2,
 'max_q': 2,
 'maxiter': 50,
 'measurement_error': False,
 'method': 'lbfgs',
 'mle_regression': True,
 'n_fits': 10,
 'n_jobs': 1,
 'offset_test_args': None,
 'out_of_sample_size': 0,
 'random': False,
 'random_state': 6095,
 'scoring': 'mse',
 'scoring_args': None,
 'seasonal': True,
 'seasonal_test': 'ocsb',
 'seasonal_test_args': None,
 'simple_differencing': False,
 'sp': 12,
 'start_P': 1,
 'start_Q': 1,
 'start_p': 0,
 'start_params': None,
 'start_q': 0,
 'stationary': False,
 'stepwise': False,
 'suppress_warnings': True,
 'test': 'kpss',
 'time_varying_regression': False,
 'trace': False,
 'trend': None,
 'update_pdq': True,
 'with_intercept': True}

In [111]:
auto_arima.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                        SARIMAX Results                                        
===============================================================================================
Dep. Variable:                                       y   No. Observations:                  132
Model:             SARIMAX(2, 0, 0)x(1, 1, [1, 2], 12)   Log Likelihood                -443.970
Date:                                 Wed, 06 Nov 2024   AIC                            901.939
Time:                                         16:25:07   BIC                            921.452
Sample:                                     01-31-1949   HQIC                           909.863
                                          - 12-31-1959                                         
Covariance Type:                                   opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.1389      0.697      0.199      0.842      -1.227       1.505
ar.L1          0.6373      0.091      6.994      0.000       0.459       0.816
ar.L2          0.2329      0.090      2.595      0.009       0.057       0.409
ar.S.L12       0.9638      0.182      5.295      0.000       0.607       1.321
ma.S.L12      -1.2102      0.347     -3.485      0.000      -1.891      -0.530
ma.S.L24       0.3295      0.120      2.742      0.006       0.094       0.565
sigma2        88.2996     20.808      4.243      0.000      47.516     129.083
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):                 1.09
Prob(Q):                              0.95   Prob(JB):                         0.58
Heteroskedasticity (H):               1.56   Skew:                            -0.07
Prob(H) (two-sided):                  0.16   Kurtosis:                         3.45
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [112]:
auto_arima_fast.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                                  y   No. Observations:                  132
Model:             SARIMAX(3, 0, 0)x(0, 1, 0, 12)   Log Likelihood                -447.843
Date:                            Wed, 06 Nov 2024   AIC                            905.686
Time:                                    16:26:24   BIC                            919.623
Sample:                                01-31-1949   HQIC                           911.346
                                     - 12-31-1959                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      5.5341      2.007      2.757      0.006       1.600       9.468
ar.L1          0.7049      0.095      7.393      0.000       0.518       0.892
ar.L2          0.2574      0.131      1.968      0.049       0.001       0.514
ar.L3         -0.1434      0.107     -1.338      0.181      -0.354       0.067
sigma2       101.0969     12.818      7.887      0.000      75.974     126.220
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):                 2.83
Prob(Q):                              0.96   Prob(JB):                         0.24
Heteroskedasticity (H):               1.41   Skew:                            -0.14
Prob(H) (two-sided):                  0.29   Kurtosis:                         3.70
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [113]:
sarima111112.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                        SARIMAX Results                                        
===============================================================================================
Dep. Variable:                                       y   No. Observations:                  132
Model:             SARIMAX(1, 1, 1)x(1, 1, [1, 2], 12)   Log Likelihood                -442.065
Date:                                 Wed, 06 Nov 2024   AIC                            898.130
Time:                                         16:26:50   BIC                            917.584
Sample:                                     01-31-1949   HQIC                           906.030
                                          - 12-31-1959                                         
Covariance Type:                                   opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0008      0.068      0.012      0.991      -0.133       0.135
ar.L1         -0.3869      0.383     -1.009      0.313      -1.139       0.365
ma.L1          0.0951      0.393      0.242      0.809      -0.676       0.866
ar.S.L12       0.9983      0.146      6.831      0.000       0.712       1.285
ma.S.L12      -1.3094      1.527     -0.858      0.391      -4.302       1.683
ma.S.L24       0.3360      0.444      0.756      0.450      -0.535       1.207
sigma2        85.0860    119.551      0.712      0.477    -149.230     319.402
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):                 0.74
Prob(Q):                              0.99   Prob(JB):                         0.69
Heteroskedasticity (H):               1.64   Skew:                            -0.10
Prob(H) (two-sided):                  0.12   Kurtosis:                         3.33
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

now, if we look at the AutoARIMA models, neither of them are detrending the data (d=0). This is clearly not intuitive, for this data set, because there is a clear trend and seasonality, we will force d=1 and D=1 and we call this new model auto_arima_augmented simply meaning that the autoarima is augmented with expert opinion. 

In [114]:
auto_arima_augmented = exp.create_model('auto_arima', cross_validation=False, information_criterion='aic', start_p=0, start_q=0, d=1, D=1)

,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
Test,0.6085,0.6927,18.5277,23.9317,0.0418,0.0403,0.8966


In [115]:
# now let's compare our intuitive sarima model with the auto_arima models:
exp.compare_models([sarima111112, auto_arima, auto_arima_fast, auto_arima_augmented], cross_validation=False)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
1,Auto ARIMA,0.3764,0.4599,11.4614,15.8874,0.0239,0.0239,0.9544,32.0600
0,ARIMA,0.4774,0.5456,14.5361,18.8508,0.0323,0.0314,0.9359,1.4000
2,Auto ARIMA,0.4893,0.5365,14.8982,18.5365,0.0310,0.0309,0.9380,4.8700
3,Auto ARIMA,0.6085,0.6927,18.5277,23.9317,0.0418,0.0403,0.8966,1.8200


AutoARIMA(max_p=2, max_q=2, random_state=6095, sp=12, start_p=0, start_q=0,
          stepwise=False, suppress_warnings=True)

so, obviously the extensive auto arima model (stepwise=False) is winning in the test set, however, I will not be using it simply because it is ignoring the trend component. 

for the rest of the notebook, we will go with the auto_arima_augmented model. 

In [71]:
auto_arima_augmented.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                                  y   No. Observations:                  132
Model:             SARIMAX(1, 1, 0)x(0, 1, 0, 12)   Log Likelihood                -447.951
Date:                            Thu, 10 Oct 2024   AIC                            899.902
Time:                                    13:44:57   BIC                            905.460
Sample:                                01-31-1949   HQIC                           902.159
                                     - 12-31-1959                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.2431      0.090     -2.697      0.007      -0.420      -0.066
sigma2       108.8757     13.306      8.183      0.000      82.797     134.954
===================================================================================
Ljung-Box (L1) (Q):                   0.02   Jarque-Bera (JB):                 0.57
Prob(Q):                              0.89   Prob(JB):                         0.75
Heteroskedasticity (H):               1.47   Skew:                            -0.03
Prob(H) (two-sided):                  0.23   Kurtosis:                         3.33
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [116]:
exp.plot_model([auto_arima_augmented], plot='forecast', data_kwargs={'fh':36, 'labels':['Auto_ARIMA_Augmented']})

In [118]:
exp.plot_model([auto_arima_augmented, auto_arima], plot='forecast', data_kwargs={'fh':60, 'labels':['Auto_ARIMA_Augmented', 'Auto_ARIMA']})
# do you see how the auto_arima is missing the long term trend? that's why we need to augment the model with d=1 and D=1. This is where expert opinion comes in.

## In-sample performance metrics?

In [119]:
# recall, our forecasting horizon was 12 months.
df.index[:-12] # train set index

PeriodIndex(['1949-01', '1949-02', '1949-03', '1949-04', '1949-05', '1949-06',
             '1949-07', '1949-08', '1949-09', '1949-10',
             ...
             '1959-03', '1959-04', '1959-05', '1959-06', '1959-07', '1959-08',
             '1959-09', '1959-10', '1959-11', '1959-12'],
            dtype='period[M]', name='Month', length=132)

In [120]:
df.head()

,Passengers
Month,
1949-01,112
1949-02,118
1949-03,132
1949-04,129
1949-05,121


In [ ]:
predictions = df.copy()
predictions['y_pred']= auto_arima_augmented.predict(df.index[:-12])
predictions['residuals']= auto_arima_augmented.predict_residuals(df[['Passengers']][:-12] )
predictions

,Passengers,y_pred,residuals
Month,,,
1949-01,112,NaN,NaN
1949-02,118,111.998425,6.001575
1949-03,132,118.000214,13.999786
1949-04,129,131.999554,-2.999554
1949-05,121,129.000193,-8.000193
...,...,...,...
1960-08,606,NaN,NaN
1960-09,508,NaN,NaN
1960-10,461,NaN,NaN


In [122]:
predictions.dropna(inplace=True)
predictions

,Passengers,y_pred,residuals
Month,,,
1949-02,118,111.998425,6.001575
1949-03,132,118.000214,13.999786
1949-04,129,131.999554,-2.999554
1949-05,121,129.000193,-8.000193
1949-06,135,121.000178,13.999822
...,...,...,...
1959-08,559,557.137287,1.862713
1959-09,463,458.729407,4.270593
1959-10,407,416.784322,-9.784322


In [123]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error

In [124]:
r2_score(predictions.Passengers, predictions.y_pred)

0.9877513769724485

In [125]:
mean_absolute_percentage_error(predictions.Passengers, predictions.y_pred)

0.0400121091048372

---
## Predict Model

This function predicts Label using a trained model. When data is None, it predicts label on the holdout set.

note: so far, our best model is the ets model


In [126]:
holdout_pred = exp.predict_model(auto_arima_augmented)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,Auto ARIMA,0.6085,0.6927,18.5277,23.9317,0.0418,0.0403,0.8966


## Finalize Model

This function trains a given estimator on the entire dataset including the holdout set.

Model finalization is the last step in the experiment. This workflow will eventually lead you to the best model for use in making predictions on new and unseen data. The finalize_model() function fits the model onto the complete dataset including the test/hold-out sample. The purpose of this function is to train the model on the complete dataset before it is deployed in production.

In [127]:
final_model = exp.finalize_model(auto_arima_augmented)

In [128]:
auto_arima_augmented.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                                  y   No. Observations:                  132
Model:             SARIMAX(1, 1, 0)x(0, 1, 0, 12)   Log Likelihood                -447.951
Date:                            Wed, 06 Nov 2024   AIC                            899.902
Time:                                    16:36:58   BIC                            905.460
Sample:                                01-31-1949   HQIC                           902.159
                                     - 12-31-1959                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.2431      0.090     -2.697      0.007      -0.420      -0.066
sigma2       108.8757     13.306      8.183      0.000      82.797     134.954
===================================================================================
Ljung-Box (L1) (Q):                   0.02   Jarque-Bera (JB):                 0.57
Prob(Q):                              0.89   Prob(JB):                         0.75
Heteroskedasticity (H):               1.47   Skew:                            -0.03
Prob(H) (two-sided):                  0.23   Kurtosis:                         3.33
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

---
## Final prediciton on unseen data

The predict_model() function is also used to predict on the unseen dataset.

In [129]:
exp.plot_model(plot='train_test_split')

In [133]:
exp.plot_model(final_model, plot='forecast', data_kwargs={'fh':24, 'labels':['AUTO_ARIMA_AUGMENTED(1,1,0)(0,1,0,12)']})

In [135]:
# seeing the predictions along with lower CI and upper CI: https://github.com/pycaret/pycaret/blob/master/pycaret/time_series/forecasting/functional.py

unseen_predictions = exp.predict_model(final_model, fh=24, return_pred_int= True, coverage=[0.025, 0.975]) # 95% CI
unseen_predictions

,y_pred,lower,upper
1961-01,451.3471,428.9616,473.7326
1961-02,427.1022,400.5662,453.6383
1961-03,463.3825,433.2625,493.5024
1961-04,499.7058,466.3852,533.0264
1961-05,514.0355,477.7959,550.2752
1961-06,571.8519,532.9114,610.7924
1961-07,661.3105,619.8447,702.7763
1961-08,648.0817,604.2358,691.9276
1961-09,551.2847,505.1814,597.3880
1961-10,501.0683,452.8131,549.3235


## Save Model

This function saves the transformation pipeline and trained model object into the current working directory as a pickle file for later use.

In [136]:
exp.save_model(final_model, 'best_arima_model')

Transformation Pipeline and Model Successfully Saved


(ForecastingPipeline(steps=[('forecaster',
                             TransformedTargetForecaster(steps=[('model',
                                                                 ForecastingPipeline(steps=[('forecaster',
                                                                                             TransformedTargetForecaster(steps=[('model',
                                                                                                                                 AutoARIMA(D=1,
                                                                                                                                           d=1,
                                                                                                                                           random_state=6095,
                                                                                                                                           sp=12,
                                              

## Load model

This function loads a previously saved pipeline.



In [137]:
my_model = load_model('best_arima_model')

Transformation Pipeline and Model Successfully Loaded


In [138]:
my_model

ForecastingPipeline(steps=[('forecaster',
                            TransformedTargetForecaster(steps=[('model',
                                                                ForecastingPipeline(steps=[('forecaster',
                                                                                            TransformedTargetForecaster(steps=[('model',
                                                                                                                                AutoARIMA(D=1,
                                                                                                                                          d=1,
                                                                                                                                          random_state=6095,
                                                                                                                                          sp=12,
                                                                                                                                          start_p=0,
                                                                                                                                          start_q=0,
                                                                                                                                          suppress_warnings=True))]))]))]))])

# Done!